In [1]:
import json
from pathlib import Path

import numpy as np
import shapefile
from prcoords import gcj_wgs_bored
import shapely.geometry as sgeom

In [2]:
dirpath_shp = Path('../frykit/data/shp')
dirpath_topo = Path('../frykit/data/topo')
if not dirpath_topo.exists():
    dirpath_topo.mkdir(parents=True)

In [3]:
# 制作九段线的shp文件.
filepath_country = dirpath_shp / 'country.shp'
filepath_geojson = dirpath_shp / '100000_full.json'
with shapefile.Reader(str(filepath_country)) as reader:
    fields = reader.fields
with open(str(filepath_geojson), encoding='utf-8') as f:
    geoj = json.load(f)
geometry = geoj['features'][-1]['geometry']

filepath_nine_line = dirpath_shp / 'nine_line.shp'
with shapefile.Writer(str(filepath_nine_line)) as writer:
    writer.fields = fields[1:]
    writer.record(cn_adcode='100000', cn_name='九段线')
    writer.shape(geometry)

In [4]:
def convert_shp_from_gcj_to_wgs(filepath_gcj, filepath_wgs):
    '''将GCJ-02坐标系的shapefile文件转为WGS84坐标系.'''
    reader = shapefile.Reader(str(filepath_gcj))
    writer = shapefile.Writer(str(filepath_wgs))
    writer.fields = reader.fields[1:]
    
    for shapeRec in reader.iterShapeRecords():
        writer.record(*shapeRec.record)
        shape = shapeRec.shape
        for i in range(len(shape.points)):
            lon, lat = shape.points[i]
            lat, lon = gcj_wgs_bored((lat, lon))
            shape.points[i] = [lon, lat]
        if not sgeom.shape(shape).is_valid:
            raise ValueError('转换导致几何错误')
        writer.shape(shape)
    
    reader.close()
    writer.close()

# 火星坐标系的shapefile文件转WGS坐标系.
for filepath_gcj in dirpath_shp.glob('*.shp'):
    filepath_wgs = dirpath_topo / filepath_gcj.name
    convert_shp_from_gcj_to_wgs(filepath_gcj, filepath_wgs)
    print(filepath_wgs, 'success')

..\frykit\data\topo\city.shp success
..\frykit\data\topo\country.shp success
..\frykit\data\topo\nine_line.shp success
..\frykit\data\topo\province.shp success
